In [1]:
# import biblioteka

import pandas as pd
import numpy as np
import time
import math
from datetime import datetime
import data_load as dl

In [2]:
# ucitavanje dataseta

df = dl.load_data(13)

c:\Users\Dell\Desktop\faks\Master\PAMU\PAMU domaci 2\data_load.py:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


In [3]:
df['WeekStartDate'] = pd.to_datetime(df['date']) - pd.to_timedelta(pd.to_datetime(df['date']).dt.weekday, unit='d')


In [4]:
# popunjavanje nedostajucih vrednosti
for col in df.columns:
    print(col, df[col].isna().sum())

# onpromotion popunjavamo sa False jer tad nije bilo promocije
# sve sto je vezano za holiday ima isti broj NA vrednosti, sto oznacava da tada nije bilo praznika
# naftu mozemo da popunimo medijanom ili rolling medijanom
# transactions mozemo da popunimo sa 0 jer tada prodavnica nije radila


id 0
date 0
store_nbr 0
item_nbr 0
unit_sales 0
onpromotion 332147
holiday_type 1469736
locale 1469736
locale_name 1469736
description 1469736
transferred 1469736
dcoilwtico 590257
family 0
class 0
perishable 0
transactions 4156
city 0
state 0
WeekStartDate 0


In [5]:
df['onpromotion'] = df['onpromotion'].fillna(False)

df['holiday_type'] = df['holiday_type'].fillna('Non existing')
df['locale'] = df['locale'].fillna('Non existing')
df['locale_name'] = df['locale_name'].fillna('Non existing')
df['description'] = df['description'].fillna('Non existing')

df['transferred'] = df['transferred'].fillna(False)

nafta_medijana = df['dcoilwtico'].median(skipna=True)
df['dcoilwtico'] = df['dcoilwtico'].fillna(nafta_medijana)
df['transactions'] = df['transactions'].fillna(0)

In [6]:
df['locale_name'].unique()

array(['Non existing', 'Ecuador', 'Manta', 'Cotopaxi', 'Cuenca',
       'Libertad', 'Riobamba', 'Puyo', 'Guaranda', 'Imbabura', 'Machala',
       'Latacunga', 'El Carmen', 'Santo Domingo', 'Cayambe', 'Guayaquil',
       'Esmeraldas', 'Ambato', 'Ibarra', 'Quevedo',
       'Santo Domingo de los Tsachilas', 'Santa Elena', 'Quito', 'Loja',
       'Salinas'], dtype=object)

In [7]:
# dummy varijable

df['Latacunga_local_holiday'] = ((df['locale_name'] == 'Latacunga') & (df['locale'] == 'Local')).astype(int)
df['Cotopaxi_regional_holiday']  = ((df['locale_name'] == 'Cotopaxi') & (df['locale'] == 'Regional')).astype(int)
df['Ecuador_national_holiday']   = ((df['locale_name'] == 'Ecuador') & (df['locale'] == 'National')).astype(int)
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,holiday_type,locale,locale_name,description,...,family,class,perishable,transactions,city,state,WeekStartDate,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday
0,12764,2013-01-02,13,103501,13.0,False,Non existing,Non existing,Non existing,Non existing,...,CLEANING,3008,0,1102.0,Latacunga,Cotopaxi,2012-12-31,0,0,0
1,12765,2013-01-02,13,105576,23.0,False,Non existing,Non existing,Non existing,Non existing,...,GROCERY I,1045,0,1102.0,Latacunga,Cotopaxi,2012-12-31,0,0,0
2,12766,2013-01-02,13,106716,11.0,False,Non existing,Non existing,Non existing,Non existing,...,GROCERY I,1032,0,1102.0,Latacunga,Cotopaxi,2012-12-31,0,0,0
3,12767,2013-01-02,13,108079,1.0,False,Non existing,Non existing,Non existing,Non existing,...,GROCERY I,1030,0,1102.0,Latacunga,Cotopaxi,2012-12-31,0,0,0
4,12768,2013-01-02,13,108701,4.0,False,Non existing,Non existing,Non existing,Non existing,...,DELI,2644,1,1102.0,Latacunga,Cotopaxi,2012-12-31,0,0,0


In [8]:
df = df.drop(columns=['locale', 'locale_name', 'description'])

In [9]:
holiday_dummies = pd.get_dummies(df['holiday_type'], prefix='holiday')
df = pd.concat([df, holiday_dummies], axis=1)
location_mask = (df['Latacunga_local_holiday'] == 1) | (df['Cotopaxi_regional_holiday'] == 1) | (df['Ecuador_national_holiday'] == 1)
for col in holiday_dummies.columns:
    df[col] = df[col].where(location_mask, 0)

In [10]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,holiday_type,transferred,dcoilwtico,family,...,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Non existing,holiday_Transfer,holiday_Work Day
0,12764,2013-01-02,13,103501,13.0,False,Non existing,False,93.14,CLEANING,...,0,0,0,0,0,0,0,0,0,0
1,12765,2013-01-02,13,105576,23.0,False,Non existing,False,93.14,GROCERY I,...,0,0,0,0,0,0,0,0,0,0
2,12766,2013-01-02,13,106716,11.0,False,Non existing,False,93.14,GROCERY I,...,0,0,0,0,0,0,0,0,0,0
3,12767,2013-01-02,13,108079,1.0,False,Non existing,False,93.14,GROCERY I,...,0,0,0,0,0,0,0,0,0,0
4,12768,2013-01-02,13,108701,4.0,False,Non existing,False,93.14,DELI,...,0,0,0,0,0,0,0,0,0,0


In [11]:
print(df['date'].nunique())
print(df['WeekStartDate'].nunique())

1679
242


In [12]:
df.dtypes

id                                    int64
date                         datetime64[ns]
store_nbr                              int8
item_nbr                              int64
unit_sales                          float64
onpromotion                            bool
holiday_type                         object
transferred                            bool
dcoilwtico                          float64
family                               object
class                                 int64
perishable                            int64
transactions                        float64
city                                 object
state                                object
WeekStartDate                datetime64[ns]
Latacunga_local_holiday               int32
Cotopaxi_regional_holiday             int32
Ecuador_national_holiday              int32
holiday_Additional                    uint8
holiday_Bridge                        uint8
holiday_Event                         uint8
holiday_Holiday                 

In [13]:
if 'holiday_Work Day' in df.columns:
    df.rename(columns={
        'holiday_Work Day': 'holiday_Work_Day'
    }, inplace=True)

df_grouped = df.groupby(['WeekStartDate', 'item_nbr']).agg({
    'store_nbr': 'first',
    'unit_sales': 'sum',
    'onpromotion': 'max', # ako je bio na promociji u barem jednom danu, racunamo da je bio na promociji tokom cele nedelje
    # description necemo da uzimamo
    'transferred': 'min', # promene praznika, pomeraji se desavaju unutar nedelje najcesce (najverovatnije), i zato uzimamo min vrednost, ako bas svaki dan postoji da je pomeren praznik neka bude 1, ali uglavnom ce biti 0 i verovatno izbacujemo ovu varijablu
    #'dcoilwtico': 'mean', # prosecna cena nafte u toj nedelji
    'family': 'first', # mozda najbolje da ne uzimamo nego da koristimo kao filter, da uzmemo 1 ili 2 kategorije, zavisi koliko nam bude bilo potrebno, ima previse vrednosti
    #'class': '', # ne znamo sta je varijabla, necemo ni da uzimamo
    'perishable': 'first', # ovo je najbolje uzeti prvi jer je pretpostavka da ce za svaki item biti isti perishable, da se nece menjati na nivou itema u nedelji, a first bi trebalo da je brze od min i max npr
    'transactions': 'sum', # ovo je na nivou prodavnice, zato mislim da mozda treba i izbaciti varijablu jer ce biti ista za svaku nedelju, ali mozda moze posluziti za razlicite nedelje kada se saberu sve transakcije
    # npr neki proizvod ce se prodavati ako je bilo vise transakcija u toj nedelji nego obicno
    'city': 'first',
    'state': 'first',
    # ovo ispod, kao i nafta ne treba da se grupise po nedelji i itemu nego samo po nedelji, i onda treba da se mergeuje sa df_grouped
    #'Cotopaxi_regional_holiday': 'max',# za praznike uzimamo max jer ako je bio praznik u toj bar 1 danu to znaci da je bio u toj celoj nedelji
    #'Ecuador_national_holiday': 'max',
    #'holiday_Additional': 'max',
    #'holiday_Bridge': 'max',
    #'holiday_Event': 'max',
    #'holiday_Holiday': 'max',
    #'holiday_Transfer': 'max',
    #'holiday_Work_Day': 'max'
}).reset_index()

df_datumi = df.groupby('WeekStartDate').agg({
    'dcoilwtico': 'mean',
    'Latacunga_local_holiday': 'max',
    'Cotopaxi_regional_holiday': 'max',
    'Ecuador_national_holiday': 'max',
    'holiday_Additional': 'max',
    'holiday_Bridge': 'max',
    'holiday_Event': 'max',
    'holiday_Holiday': 'max',
    'holiday_Transfer': 'max',
    'holiday_Work_Day': 'max'
})
df_grouped = df_grouped.merge(df_datumi, on='WeekStartDate', how='left')

In [14]:
# provera da li ne postoje 2 cene nafte u jednoj nedelji
# dobro je jer smo kao rezultat dobili praznu seriju
df_oil_check = df_grouped.groupby('WeekStartDate')['dcoilwtico'].nunique()
print(df_oil_check[df_oil_check > 1])

Series([], Name: dcoilwtico, dtype: int64)


In [15]:
df_grouped.shape

(347196, 21)

In [16]:
df_grouped.head()

,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,city,...,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day
0,2012-12-31,103501,13,32.0,False,False,CLEANING,0,5241.0,Latacunga,...,75.654865,0,0,1,0,0,0,0,0,1
1,2012-12-31,103520,13,5.0,False,False,GROCERY I,0,3223.0,Latacunga,...,75.654865,0,0,1,0,0,0,0,0,1
2,2012-12-31,105576,13,81.0,False,False,GROCERY I,0,5241.0,Latacunga,...,75.654865,0,0,1,0,0,0,0,0,1
3,2012-12-31,105857,13,10.0,False,False,GROCERY I,0,1336.0,Latacunga,...,75.654865,0,0,1,0,0,0,0,0,1
4,2012-12-31,106716,13,40.0,False,False,GROCERY I,0,5241.0,Latacunga,...,75.654865,0,0,1,0,0,0,0,0,1


In [17]:
# krairanje izlazne varijable
# ako postoji u datasetu znaci da je poruceno u toj nedelji
df_grouped['Ordered'] = 1

# sada je potrebno kreirati sve kombinacije u svakoj nedelji i kolonu Ordered popuniti nulom jer to predstavlja da se taj proizvod u toj nedelji nije porucio

In [18]:
print(df_grouped['WeekStartDate'].nunique())
print(df_grouped['item_nbr'].nunique())
# ukupno 618.068 redova treba da ima kompletan dataset

242
2554


In [19]:
# 1. Dobavi sve unikatne vrednosti WeekStartDate i item_nbr
sve_nedelje = df_grouped['WeekStartDate'].unique()
svi_proizvodi = df_grouped['item_nbr'].unique()

# 2. Napravi sve moguće kombinacije (proizvod × nedelja)
import itertools

sve_kombinacije = pd.DataFrame(list(itertools.product(sve_nedelje, svi_proizvodi)),
                                columns=['WeekStartDate', 'item_nbr'])

# 3. Spoji originalni df sa ovim novim kombinacijama
df_popunjen = sve_kombinacije.merge(df_grouped, on=['WeekStartDate', 'item_nbr'], how='left')

# 4. Gde god nedostaje Ordered, postavi na 0
df_popunjen['Ordered'] = df_popunjen['Ordered'].fillna(0)

In [20]:
df_popunjen.shape

(618068, 22)

In [21]:
pd.set_option('display.max_columns', None)
df_popunjen.head()

,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,city,state,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day,Ordered
0,2012-12-31,103501,13.0,32.0,False,False,CLEANING,0.0,5241.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,2012-12-31,103520,13.0,5.0,False,False,GROCERY I,0.0,3223.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2012-12-31,105576,13.0,81.0,False,False,GROCERY I,0.0,5241.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,2012-12-31,105857,13.0,10.0,False,False,GROCERY I,0.0,1336.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,2012-12-31,106716,13.0,40.0,False,False,GROCERY I,0.0,5241.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [22]:
for col in df_popunjen.columns:
    print(col, df_popunjen[col].isna().sum())

WeekStartDate 0
item_nbr 0
store_nbr 270872
unit_sales 270872
onpromotion 270872
transferred 270872
family 270872
perishable 270872
transactions 270872
city 270872
state 270872
dcoilwtico 270872
Latacunga_local_holiday 270872
Cotopaxi_regional_holiday 270872
Ecuador_national_holiday 270872
holiday_Additional 270872
holiday_Bridge 270872
holiday_Event 270872
holiday_Holiday 270872
holiday_Transfer 270872
holiday_Work_Day 270872
Ordered 0


Popunjavanje nedostajucih vrednosti (vestacki popunjen dataset):

1. store_nbr - ffill i bfill (svuda je isto)
2. unit_sales - 0 jer tada nista nije bilo prodato
3. onpromotion - ffill i bfill ali kada se grupise i grupise po datumu i itemu
4. transferred - ffill i bfill kada se grupise po datumu
5. dcoilwtico - ffill i bfill kada se grupise po datumu
6. family - ffill i bfill kada se grupise po itemu
7. transactions - ffill i bfill kada se grupise po datumu
8. city i state - ffill i bfill (svuda je isto)
9. svi praznici - ffill i bfill kada se grupise po datumu
10. perishable - ffill i bfill kada se grupise po itemu

ffill i bfill idu zajedno jer kod ffill prvi red ostaje null a kod bfill poslednji red ostaje null (ukoliko su vec bili null)

In [23]:
# samo ffill i bfill
for col in ['store_nbr', 'city', 'state']:
    df_popunjen[col] = df_popunjen[col].ffill().bfill()


In [24]:
# unit_sales
df_popunjen['unit_sales'] = df_popunjen['unit_sales'].fillna(0)

# grupisanje po datumu
for col in ['transferred', 'dcoilwtico', 'transactions', 
            'Cotopaxi_regional_holiday', 'Ecuador_national_holiday', 'holiday_Additional', 
            'holiday_Bridge', 'holiday_Event', 'holiday_Holiday',
            'holiday_Transfer', 'holiday_Work_Day']:
    df_popunjen[col] = df_popunjen.groupby('WeekStartDate')[col].ffill().bfill()

In [25]:
# po itemu
df_popunjen['family'] = df_popunjen.groupby('item_nbr')['family'].ffill().bfill()
df_popunjen['perishable'] = df_popunjen.groupby('item_nbr')['perishable'].ffill().bfill()

# po datumu i itemu
df_popunjen['onpromotion'] = df_popunjen.groupby(['WeekStartDate', 'item_nbr'])['onpromotion'].ffill().bfill()

# vise nema nigde nedostajucih vrednosti

In [26]:
df_popunjen = df_popunjen.sort_values(['WeekStartDate', 'item_nbr'])
df_popunjen.head()

,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,city,state,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day,Ordered
0,2012-12-31,103501,13.0,32.0,False,False,CLEANING,0.0,5241.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,2012-12-31,103520,13.0,5.0,False,False,GROCERY I,0.0,3223.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2012-12-31,105576,13.0,81.0,False,False,GROCERY I,0.0,5241.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1577,2012-12-31,105693,13.0,0.0,False,False,CLEANING,0.0,4362.0,Latacunga,Cotopaxi,75.654865,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2012-12-31,105857,13.0,10.0,False,False,GROCERY I,0.0,1336.0,Latacunga,Cotopaxi,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [27]:
df_popunjen = df_popunjen.drop(columns=['city', 'state'])

In [28]:
# jedan mesec ima 4 nedelje, 3 meseca imaju ukupno 12 nedelja
# lagovi za Ordered
df_popunjen = df_popunjen.sort_values(['item_nbr', 'WeekStartDate'])
for lag in range(1,13):
    col_name = f"Ordered_lag_{lag}"
    df_popunjen[col_name] = df_popunjen.groupby('item_nbr')['Ordered'].shift(lag).fillna(0)


In [29]:
df_popunjen = df_popunjen.sort_values(['item_nbr', 'WeekStartDate'])
df_popunjen.head(15)

,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day,Ordered,Ordered_lag_1,Ordered_lag_2,Ordered_lag_3,Ordered_lag_4,Ordered_lag_5,Ordered_lag_6,Ordered_lag_7,Ordered_lag_8,Ordered_lag_9,Ordered_lag_10,Ordered_lag_11,Ordered_lag_12
0,2012-12-31,103501,13.0,32.0,False,False,CLEANING,0.0,5241.0,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2554,2013-01-07,103501,13.0,27.0,False,False,CLEANING,0.0,6370.0,80.543340,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5108,2013-01-14,103501,13.0,26.0,False,False,CLEANING,0.0,6191.0,81.228634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7662,2013-01-21,103501,13.0,27.0,False,False,CLEANING,0.0,6187.0,75.488994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10216,2013-01-28,103501,13.0,39.0,False,False,CLEANING,0.0,6513.0,83.011750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12770,2013-02-04,103501,13.0,21.0,False,False,CLEANING,0.0,5834.0,82.948236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15324,2013-02-11,103501,13.0,21.0,False,False,CLEANING,0.0,6574.0,82.899848,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17878,2013-02-18,103501,13.0,26.0,False,False,CLEANING,0.0,6576.0,74.763149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
20432,2013-02-25,103501,13.0,23.0,False,False,CLEANING,0.0,6765.0,79.024500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
22986,2013-03-04,103501,13.0,27.0,False,False,CLEANING,0.0,6936.0,78.935561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
# lag za unit sales
# lag za dcoilwtico
df_popunjen = df_popunjen.sort_values(['item_nbr', 'WeekStartDate'])
for lag in range(1,13):
    col_name = f"unit_sales_lag_{lag}"
    df_popunjen[col_name] = df_popunjen.groupby(['item_nbr'])['unit_sales'].shift(lag).fillna(0)


df_popunjen['WeekStartDate'] = pd.to_datetime(df_popunjen['WeekStartDate'])

df_oil = df_popunjen[['WeekStartDate', 'dcoilwtico']].drop_duplicates().sort_values('WeekStartDate')

df_oil = (
    df_popunjen[['WeekStartDate', 'dcoilwtico']]
    .drop_duplicates()
    .sort_values('WeekStartDate')
    .reset_index(drop=True)
)
for lag in range(1, 13):
    df_oil[f'dcoilwtico_lag_{lag}'] = df_oil['dcoilwtico'].shift(lag).fillna(0)

#print(df_oil.head())
# Pretvori u dict po WeekStartDate i mapiraj
for lag in range(1, 13):
    lag_col = f'dcoilwtico_lag_{lag}'
    mapping = pd.Series(df_oil[lag_col].values, index=df_oil['WeekStartDate'])
    df_popunjen[lag_col] = df_popunjen['WeekStartDate'].map(mapping)

df_oil = df_oil.sort_values('WeekStartDate')  # obavezno sortiranje

rolling_windows = [4, 8, 12]

for win in rolling_windows:
    df_oil[f'dcoilwtico_rolling_mean_{win}'] = (
        df_oil['dcoilwtico'].rolling(window=win, min_periods=1).mean()
    )



In [46]:

for win in rolling_windows:
    roll_col = f'dcoilwtico_rolling_mean_{win}'
    mapping = pd.Series(df_oil[roll_col].values, index=df_oil['WeekStartDate'])
    df_popunjen[roll_col] = df_popunjen['WeekStartDate'].map(mapping)

In [47]:
df_popunjen.head(20)

,index,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day,Ordered,Ordered_lag_1,Ordered_lag_2,Ordered_lag_3,Ordered_lag_4,Ordered_lag_5,Ordered_lag_6,Ordered_lag_7,Ordered_lag_8,Ordered_lag_9,Ordered_lag_10,Ordered_lag_11,Ordered_lag_12,unit_sales_lag_1,unit_sales_lag_2,unit_sales_lag_3,unit_sales_lag_4,unit_sales_lag_5,unit_sales_lag_6,unit_sales_lag_7,unit_sales_lag_8,unit_sales_lag_9,unit_sales_lag_10,unit_sales_lag_11,unit_sales_lag_12,dcoilwtico_lag_1,dcoilwtico_lag_2,dcoilwtico_lag_3,dcoilwtico_lag_4,dcoilwtico_lag_5,dcoilwtico_lag_6,dcoilwtico_lag_7,dcoilwtico_lag_8,dcoilwtico_lag_9,dcoilwtico_lag_10,dcoilwtico_lag_11,dcoilwtico_lag_12,Latacunga_local_holiday_lag_1,Cotopaxi_regional_holiday_lag_1,Ecuador_national_holiday_lag_1,holiday_Additional_lag_1,holiday_Bridge_lag_1,holiday_Event_lag_1,holiday_Holiday_lag_1,holiday_Transfer_lag_1,holiday_Work_Day_lag_1,Ordered_rolling_mean_4,Ordered_rolling_mean_8,Ordered_rolling_mean_12,unit_sales_rolling_mean_4,unit_sales_rolling_mean_8,unit_sales_rolling_mean_12,WeekNumber,sin_0.5_pi,cos_0.5_pi,sin_1_pi,cos_1_pi,sin_2_pi,cos_2_pi,sin_4_pi,cos_4_pi,dcoilwtico_rolling_mean_4,dcoilwtico_rolling_mean_8,dcoilwtico_rolling_mean_12
0,0,2012-12-31,103501,13.0,32.0,False,False,CLEANING,0.0,5241.0,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,32.000000,32.000000,32.000000,1,0.030203,0.999544,0.060378,0.998176,0.120537,0.992709,0.239316,0.970942,75.654865,75.654865,75.654865
1,2554,2013-01-07,103501,13.0,27.0,False,False,CLEANING,0.0,6370.0,80.543340,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,29.500000,29.500000,29.500000,2,0.060378,0.998176,0.120537,0.992709,0.239316,0.970942,0.464723,0.885456,78.099102,78.099102,78.099102
2,5108,2013-01-14,103501,13.0,26.0,False,False,CLEANING,0.0,6191.0,81.228634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.543340,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,28.333333,28.333333,28.333333,3,0.090499,0.995897,0.180255,0.98362,0.354605,0.935016,0.663123,0.748511,79.142280,79.142280,79.142280
3,7662,2013-01-21,103501,13.0,27.0,False,False,CLEANING,0.0,6187.0,75.488994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.228634,80.543340,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,28.000000,28.000000,28.000000,4,0.120537,0.992709,0.239316,0.970942,0.464723,0.885456,0.822984,0.568065,78.228958,78.228958,78.228958
4,10216,2013-01-28,103501,13.0,39.0,False,False,CLEANING,0.0,6513.0,83.011750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,26.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.488994,81.228634,80.543340,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,29.750000,30.200000,30.200000,5,0.150465,0.988615,0.297503,0.954721,0.568065,0.822984,0.935016,0.354605,80.068180,79.185517,79.185517
5,12770,2013-02-04,103501,13.0,21.0,False,F

In [32]:
df_popunjen['dcoilwtico'].nunique()

242

In [33]:
# sve kolone koje sadrze holiday lag 1
df_popunjen = df_popunjen.sort_values('WeekStartDate')

# Pronađi sve kolone koje sadrže 'holiday' u imenu
holiday_cols = [col for col in df_popunjen.columns if 'holiday' in col]

# Napravi lag_1 kolone (bez groupby)
for col in holiday_cols:
    lag_col = f"{col}_lag_1"
    df_popunjen[lag_col] = df_popunjen[col].shift(1).fillna(0)

In [34]:
df_popunjen.head()

,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day,Ordered,Ordered_lag_1,Ordered_lag_2,Ordered_lag_3,Ordered_lag_4,Ordered_lag_5,Ordered_lag_6,Ordered_lag_7,Ordered_lag_8,Ordered_lag_9,Ordered_lag_10,Ordered_lag_11,Ordered_lag_12,unit_sales_lag_1,unit_sales_lag_2,unit_sales_lag_3,unit_sales_lag_4,unit_sales_lag_5,unit_sales_lag_6,unit_sales_lag_7,unit_sales_lag_8,unit_sales_lag_9,unit_sales_lag_10,unit_sales_lag_11,unit_sales_lag_12,dcoilwtico_lag_1,dcoilwtico_lag_2,dcoilwtico_lag_3,dcoilwtico_lag_4,dcoilwtico_lag_5,dcoilwtico_lag_6,dcoilwtico_lag_7,dcoilwtico_lag_8,dcoilwtico_lag_9,dcoilwtico_lag_10,dcoilwtico_lag_11,dcoilwtico_lag_12,Latacunga_local_holiday_lag_1,Cotopaxi_regional_holiday_lag_1,Ecuador_national_holiday_lag_1,holiday_Additional_lag_1,holiday_Bridge_lag_1,holiday_Event_lag_1,holiday_Holiday_lag_1,holiday_Transfer_lag_1,holiday_Work_Day_lag_1
0,2012-12-31,103501,13.0,32.0,False,False,CLEANING,0.0,5241.0,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2550,2012-12-31,2056764,13.0,0.0,False,False,CLEANING,0.0,4362.0,75.654865,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
889,2012-12-31,1117661,13.0,0.0,False,False,CLEANING,0.0,4362.0,75.654865,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
853,2012-12-31,1047699,13.0,0.0,False,False,CLEANING,0.0,4362.0,75.654865,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
65,2012-12-31,169028,13.0,5.0,False,False,GROCERY I,0.0,3131.0,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
df_popunjen['WeekStartDate'].min()
df_popunjen['WeekStartDate'].max()

Timestamp('2017-08-14 00:00:00')

In [36]:
# dodavanje rolling means kolona u dataset

# dodaju se na isti nacin kao lag kolone, samo sto se umesto shift koristi rolling funkcija
# grupisanje je isto kao za lag kolone, veoma bitna stvar

df_popunjen = df_popunjen.sort_values(['item_nbr', 'WeekStartDate'])

rolling_windows = [4, 8, 12]
target_cols = ['Ordered', 'unit_sales']

for col in target_cols:
    for win in rolling_windows:
        new_col = f"{col}_rolling_mean_{win}"
        df_popunjen[new_col] = (
            df_popunjen
            .groupby('item_nbr')[col]
            .transform(lambda x: x.rolling(window=win, min_periods=1).mean())
        )



In [40]:
df_popunjen = df_popunjen.reset_index()

In [42]:
df_popunjen.head()

,index,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day,Ordered,Ordered_lag_1,Ordered_lag_2,Ordered_lag_3,Ordered_lag_4,Ordered_lag_5,Ordered_lag_6,Ordered_lag_7,Ordered_lag_8,Ordered_lag_9,Ordered_lag_10,Ordered_lag_11,Ordered_lag_12,unit_sales_lag_1,unit_sales_lag_2,unit_sales_lag_3,unit_sales_lag_4,unit_sales_lag_5,unit_sales_lag_6,unit_sales_lag_7,unit_sales_lag_8,unit_sales_lag_9,unit_sales_lag_10,unit_sales_lag_11,unit_sales_lag_12,dcoilwtico_lag_1,dcoilwtico_lag_2,dcoilwtico_lag_3,dcoilwtico_lag_4,dcoilwtico_lag_5,dcoilwtico_lag_6,dcoilwtico_lag_7,dcoilwtico_lag_8,dcoilwtico_lag_9,dcoilwtico_lag_10,dcoilwtico_lag_11,dcoilwtico_lag_12,Latacunga_local_holiday_lag_1,Cotopaxi_regional_holiday_lag_1,Ecuador_national_holiday_lag_1,holiday_Additional_lag_1,holiday_Bridge_lag_1,holiday_Event_lag_1,holiday_Holiday_lag_1,holiday_Transfer_lag_1,holiday_Work_Day_lag_1,Ordered_rolling_mean_4,Ordered_rolling_mean_8,Ordered_rolling_mean_12,unit_sales_rolling_mean_4,unit_sales_rolling_mean_8,unit_sales_rolling_mean_12
0,0,2012-12-31,103501,13.0,32.0,False,False,CLEANING,0.0,5241.0,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,32.000000,32.000000,32.000000
1,2554,2013-01-07,103501,13.0,27.0,False,False,CLEANING,0.0,6370.0,80.543340,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.654865,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,29.500000,29.500000,29.500000
2,5108,2013-01-14,103501,13.0,26.0,False,False,CLEANING,0.0,6191.0,81.228634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.543340,75.654865,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,28.333333,28.333333,28.333333
3,7662,2013-01-21,103501,13.0,27.0,False,False,CLEANING,0.0,6187.0,75.488994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.228634,80.543340,75.654865,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,28.000000,28.000000,28.000000
4,10216,2013-01-28,103501,13.0,39.0,False,False,CLEANING,0.0,6513.0,83.011750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,26.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.488994,81.228634,80.543340,75.654865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,29.750000,30.200000,30.200000


In [43]:
# dodavanja funkcija za sezonalnost (sin i cos, fourier transformacija)

# prvo se dodaje WeekNumber koji se krece od 1 do 52 (broj nedelje u godini)
df_popunjen['WeekNumber'] = df_popunjen['WeekStartDate'].dt.isocalendar().week

uglovi = [0.5, 1, 2, 4] # sa koliko mnozimo pi

for ugao in uglovi:
    sin_col = f"sin_{ugao}_pi"
    cos_col = f"cos_{ugao}_pi"

    df_popunjen[sin_col] = np.sin(ugao * np.pi * df_popunjen['WeekNumber'] / 52) # / 52 jer ima 52 nedelje u godini
    df_popunjen[cos_col] = np.cos(ugao * np.pi * df_popunjen['WeekNumber'] / 52) # / 52 jer ima 52 nedelje u godini

In [45]:
df_popunjen.head(20)

,index,WeekStartDate,item_nbr,store_nbr,unit_sales,onpromotion,transferred,family,perishable,transactions,dcoilwtico,Latacunga_local_holiday,Cotopaxi_regional_holiday,Ecuador_national_holiday,holiday_Additional,holiday_Bridge,holiday_Event,holiday_Holiday,holiday_Transfer,holiday_Work_Day,Ordered,Ordered_lag_1,Ordered_lag_2,Ordered_lag_3,Ordered_lag_4,Ordered_lag_5,Ordered_lag_6,Ordered_lag_7,Ordered_lag_8,Ordered_lag_9,Ordered_lag_10,Ordered_lag_11,Ordered_lag_12,unit_sales_lag_1,unit_sales_lag_2,unit_sales_lag_3,unit_sales_lag_4,unit_sales_lag_5,unit_sales_lag_6,unit_sales_lag_7,unit_sales_lag_8,unit_sales_lag_9,unit_sales_lag_10,unit_sales_lag_11,unit_sales_lag_12,dcoilwtico_lag_1,dcoilwtico_lag_2,dcoilwtico_lag_3,dcoilwtico_lag_4,dcoilwtico_lag_5,dcoilwtico_lag_6,dcoilwtico_lag_7,dcoilwtico_lag_8,dcoilwtico_lag_9,dcoilwtico_lag_10,dcoilwtico_lag_11,dcoilwtico_lag_12,Latacunga_local_holiday_lag_1,Cotopaxi_regional_holiday_lag_1,Ecuador_national_holiday_lag_1,holiday_Additional_lag_1,holiday_Bridge_lag_1,holiday_Event_lag_1,holiday_Holiday_lag_1,holiday_Transfer_lag_1,holiday_Work_Day_lag_1,Ordered_rolling_mean_4,Ordered_rolling_mean_8,Ordered_rolling_mean_12,unit_sales_rolling_mean_4,unit_sales_rolling_mean_8,unit_sales_rolling_mean_12,WeekNumber,sin_0.5_pi,cos_0.5_pi,sin_1_pi,cos_1_pi,sin_2_pi,cos_2_pi,sin_4_pi,cos_4_pi
0,0,2012-12-31,103501,13.0,32.0,False,False,CLEANING,0.0,5241.0,75.654865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,32.000000,32.000000,32.000000,1,0.030203,0.999544,0.060378,0.998176,0.120537,0.992709,0.239316,0.970942
1,2554,2013-01-07,103501,13.0,27.0,False,False,CLEANING,0.0,6370.0,80.543340,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,29.500000,29.500000,29.500000,2,0.060378,0.998176,0.120537,0.992709,0.239316,0.970942,0.464723,0.885456
2,5108,2013-01-14,103501,13.0,26.0,False,False,CLEANING,0.0,6191.0,81.228634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.543340,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,28.333333,28.333333,28.333333,3,0.090499,0.995897,0.180255,0.98362,0.354605,0.935016,0.663123,0.748511
3,7662,2013-01-21,103501,13.0,27.0,False,False,CLEANING,0.0,6187.0,75.488994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.228634,80.543340,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,28.000000,28.000000,28.000000,4,0.120537,0.992709,0.239316,0.970942,0.464723,0.885456,0.822984,0.568065
4,10216,2013-01-28,103501,13.0,39.0,False,False,CLEANING,0.0,6513.0,83.011750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,26.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.488994,81.228634,80.543340,75.654865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,29.750000,30.200000,30.200000,5,0.150465,0.988615,0.297503,0.954721,0.568065,0.822984,0.935016,0.354605
5,12770,2013-02-04,103501,13.0,21.0,False,False,CLEANING,0.0,5834.0,82.948236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,27.0,26.0,27.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.011750,75.488994,81.228634,80.543340,75.654865,0.0